In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error

In [2]:
path = 'C:/Users/user/Desktop/PLoSONE/data/'

data = pd.read_csv(path + 'dataset_household_PLoS ONE.csv')
data.head()

,Date,Hour_x,Hour_y,DOTW_x,DOTW_y,Holi,Temp,Humi,WS,THI,WCT,Cons_1,Holi_1,Cons_7,Holi_7,Cons_avg,Consumption
0,2016-01-11 18:00,-1.000000,-1.840000e-16,0.781831,0.62349,0,5.9,92.0,5.0,43.29672,5.103854,0,0,0,0,0.0,330
1,2016-01-11 19:00,-0.965926,2.588190e-01,0.781831,0.62349,0,6.0,91.0,6.0,43.55240,4.872989,0,0,0,0,0.0,1430
2,2016-01-11 20:00,-0.866025,5.000000e-01,0.781831,0.62349,0,6.0,88.0,6.0,43.80320,4.872989,0,0,0,0,0.0,690
3,2016-01-11 21:00,-0.707107,7.071068e-01,0.781831,0.62349,0,6.0,87.0,6.0,43.88680,4.872989,0,0,0,0,0.0,780
4,2016-01-11 22:00,-0.500000,8.660254e-01,0.781831,0.62349,0,5.6,88.0,6.0,43.13072,4.413133,0,0,0,0,0.0,620


In [3]:
data_drop = data.drop(columns = 'Date', axis = 1)
data_drop.head()

,Hour_x,Hour_y,DOTW_x,DOTW_y,Holi,Temp,Humi,WS,THI,WCT,Cons_1,Holi_1,Cons_7,Holi_7,Cons_avg,Consumption
0,-1.000000,-1.840000e-16,0.781831,0.62349,0,5.9,92.0,5.0,43.29672,5.103854,0,0,0,0,0.0,330
1,-0.965926,2.588190e-01,0.781831,0.62349,0,6.0,91.0,6.0,43.55240,4.872989,0,0,0,0,0.0,1430
2,-0.866025,5.000000e-01,0.781831,0.62349,0,6.0,88.0,6.0,43.80320,4.872989,0,0,0,0,0.0,690
3,-0.707107,7.071068e-01,0.781831,0.62349,0,6.0,87.0,6.0,43.88680,4.872989,0,0,0,0,0.0,780
4,-0.500000,8.660254e-01,0.781831,0.62349,0,5.6,88.0,6.0,43.13072,4.413133,0,0,0,0,0.0,620


In [4]:
x_train = data_drop.iloc[:2310]
x_test = data_drop.iloc[2310:]
y_train = data_drop.iloc[:2310]

In [5]:
# target 정의 
y = y_train['Consumption']

In [6]:
# 데이터 셋 분리
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y, test_size=0.3, random_state=42)

In [8]:
# xgb
model = XGBRegressor()

params = {
    'n_estimators': [250, 500, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [6, 8, 10],
    'subsample': [0.5, 0.75, 1.0],
    'colsample_bytree': [0.5, 0.75, 1.0],
    'booster': ['gbtree', 'dart']
         }

# GridSEarch
gr = GridSearchCV(model, param_grid = params, cv = 5)
gr.fit(X_train, Y_train)

print(gr.best_params_)

KeyboardInterrupt: 

In [8]:
xgb = XGBRegressor(random_state=42) # 다시 넣기
xgb.fit(X_train, Y_train)

y_pred = xgb.predict(X_valid)
y_pred

array([ 330.00181948,  369.96595259,  539.99149404,  222.32179841,
        559.94314052,  279.99659255, 1110.78058546,  329.98815302,
        379.95621884,  319.97761921,  449.60873146,  806.00870606,
        489.98691941,  409.88530185,  429.61814518,  680.09145585,
        819.70103713,  780.20641541,  909.63302842,  690.46588288,
       1008.33868886,  279.99270318,  290.73666517,  359.17081457,
       2068.40385248,  609.90324819,  519.91897932,  351.1232756 ,
        270.05509805,  808.08786388,  300.01872162,  279.99600392,
        279.97662587,  310.01748943,  359.94986589,  389.96401786,
        329.94038058,  279.88549334,  219.75116066,  439.88048155,
       2081.20408583,  319.53127135,  529.97139517,  279.98763108,
        345.79448401,  280.06245821,  435.74386868,  270.08518456,
        389.97749441,  208.73153007,  280.03391543,  956.54209648,
        350.10980234,  279.94290292,  330.02536015,  379.9529555 ,
        609.93467563,  329.99294254,  699.6023453 ,  250.18263

In [29]:
# 실제값 
y_predict = xgb.predict(x_test).astype(int)
y_predict.shape

(979,)

In [30]:
y_predict

array([ 339,  300,  329,  300,  339,  289,  300,  329,  350,  890,  679,
       2742,  790,  639,  420,  450,  599,  978, 1470,  699,  729,  550,
        420,  319,  339,  339,  409,  319,  250,  319,  310,  330,  289,
        330,  300,  289,  490,  310,  299,  250,  349,  440, 1763,  650,
        710,  730,  289,  269,  310,  320,  230,  310,  319,  310,  679,
        459,  780,  310,  300,  240,  310,  309,  249,  319,  289,  340,
       1610,  698,  680,  499,  329,  310,  229,  329,  310,  240,  310,
        319,  280,  420,  330,  350,  350,  389,  249,  309,  350, 1045,
        465,  425, 1367, 1433,  710,  629,  409,  319,  229,  319,  329,
        300,  240,  320,  420,  330,  280,  320,  310,  300,  329,  315,
        385,  365,  435,  494, 1257, 1113,  714, 1027,  427,  399,  419,
        349,  389,  409,  389,  409,  470,  369, 1008, 1650, 1891, 2356,
       1555, 2230, 1435,  500, 2169, 1310, 2783,  849,  609,  640,  379,
        340,  320,  339,  300,  340,  300,  340,  3

In [32]:
df = pd.DataFrame({'Date' : data['Date'].iloc[2310:], 'Actual' : data['Consumption'].iloc[2310:], 'Predict' : y_predict}).reset_index(drop=True)
df.to_csv('C:/Users/user/Desktop/PLoSONE/predict/household_PLoS_XGB.csv', header=True, index=False)